In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset, random_split, WeightedRandomSampler
from torchvision.datasets import ImageFolder
from tqdm import tqdm
import numpy as np
import os
from google.colab import drive
from collections import Counter
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/MyDrive/TR_DIMA/Logit_compensation/')
from logitadjust import LogitAdjust


# Path to the dataset and model
path_train_dataset = "/content/drive/MyDrive/TR_DIMA/training_set_reduit"

# Transformations for the dataset
base_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),  # Convert images to PyTorch tensors
])

augmented_transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.GaussianBlur(kernel_size=9, sigma=(0.01, 5)),
        transforms.RandomResizedCrop(size=224, scale=(0.7, 1.0), ratio = (0.2,5)),
        transforms.ToTensor(),
    ])

Mounted at /content/drive


In [ ]:
# Load the training dataset

dataset_init = datasets.ImageFolder(root=path_train_dataset, transform=augmented_transform)

train_size = int(0.8 * len(dataset_init))
valid_size = len(dataset_init) - train_size

train_dataset, valid_dataset = random_split(dataset_init, [train_size, valid_size])

# Create DataLoaders for training and validation datasets

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=10, pin_memory=True)
val_loader = DataLoader(valid_dataset, batch_size=128, shuffle=False, num_workers=10, pin_memory=True)

valid_classes = dataset_init.classes

In [ ]:
num_all = Counter(dataset_init.targets)
distrib = [num_all[i] for i in range(0,13)]

In [33]:
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, len(valid_classes))

# Device config
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the device
model.to(device)

# Define the loss function and optimizer
criterion = LogitAdjust(distrib,tau=1.9,weight=None)
optimizer = optim.Adam(model.parameters(), lr=1e-5)

In [34]:
# Training function

num_epochs = 100
patience = 5
early_stopping_counter = 0
best_val_loss = float('inf')
gradients = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]", leave=False)

    for inputs, labels in train_bar:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        train_bar.set_postfix(loss=loss.item())

    train_loss = running_loss / len(train_loader.dataset)

    # Validation
    model.eval()
    val_loss = 0.0
    val_bar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]", leave=False)

    with torch.no_grad():
        for inputs, labels in val_bar:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            val_bar.set_postfix(loss=loss.item())

    val_loss = val_loss / len(val_loader.dataset)

    print(f"Epoch {epoch+1}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}")

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(model.state_dict(), "/content/drive/MyDrive/TR_DIMA/Entrainement/best_model_logit_adjustment_1_9.pth")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping déclenché.")
            break


Epoch 1: Train Loss = 0.5203, Val Loss = 0.2648


Epoch 2: Train Loss = 0.2188, Val Loss = 0.1812


Epoch 3: Train Loss = 0.1660, Val Loss = 0.1502


Epoch 4: Train Loss = 0.1446, Val Loss = 0.1361


Epoch 5: Train Loss = 0.1276, Val Loss = 0.1329


...

Epoch 9: Train Loss = 0.0914, Val Loss = 0.1053


Epoch 10: Train Loss = 0.0863, Val Loss = 0.1078


Epoch 11: Train Loss = 0.0803, Val Loss = 0.1035


Epoch 12: Train Loss = 0.0767, Val Loss = 0.1049


Epoch 13: Train Loss = 0.0719, Val Loss = 0.1035
Early stopping déclenché.
